In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client, LocalCluster

client = Client() # Note that `memory_limit` is the limit **per worker**.
# n_workers=4,
#                 threads_per_worker=1,
#                 memory_limit='3GB'
client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

In [ ]:
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import scipy.constants
import scipy.signal as signal
from bokeh.io import export_svg as b_export_svg
import pickle
import holoviews as hv

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

sys.path.append("../../preprocessing/")
from generate_chirp import generate_chirp

from selenium import webdriver
import chromedriver_binary  # Adds chromedriver binary to path

In [ ]:
def export_svg(obj, filename):
    plot_state = hv.renderer('bokeh').get_plot(obj).state
    plot_state.output_backend = 'svg'
    b_export_svg(plot_state, filename=filename)

### Open and resave file

In [ ]:
# file path to data and configs
#prefix = "/Users/abroome/Documents/SDR/uhd_radar/data/20230710_095333"
#prefix = "/Users/abroome/Desktop/McMurdo2022/11222022_anna/20221122_135614"
#prefix = "/home/radioglaciolgy/anna/uhd_radar/data/20230719_103400"
#prefix = "/Volumes/Extreme SSD/Summit2023/07232023_anna/20230723_151432"
#prefix = "/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_161051"
#prefix = "/Volumes/Extreme SSD/Summit2023/07292023_anna/20230729_154922"

#prefix = "/Volumes/T7 Shield/Summit2023/08022023_anna/20230802_152434"
#prefix = "/Volumes/T7 Shield/Summit2023/08042023_anna/20230804_155432"
#prefix = "/Volumes/T7 Shield/Summit2023/08082023_anna/20230808_164554"
#prefix = "/Volumes/T7 Shield/Summit2023/08082023_anna/20230808_181237"
prefix = "/Volumes/T7 Shield/Summit2023/07252023_anna/20230725_161051"

#prefix = "/Volumes/T7 Shield/Summit2023/08022023_anna/20230802_152434"
#prefix = "/Volumes/T7 Shield/Summit2023/08042023_anna/20230804_154502"
#prefix = "/Volumes/T7 Shield/Summit2023/08042023_anna/20230804_155432"

#prefix = "/Volumes/T7 Shield/Summit2023/08042023_anna/20230804_161102"

#prefix = "/Volumes/Extreme SSD/lab_debugging/prf_data/raw/20231103_091920"
#prefix = "/Volumes/Extreme SSD/lab_debugging/prf_data/raw/20231103_090157"

# resave data as zarr for dask processing
zarr_path = pr.save_radar_data_to_zarr(prefix)#, zarr_base_location="/Volumes/Extreme SSD/Summit2023/07232023_anna/zarrs/")

# open zarr file, adjust chunk size to be 10 MB - 1 GB based on sample rate/bit depth
raw = xr.open_zarr(zarr_path, chunks={"pulse_idx": 1000})
#raw = pr.invert_phase_dithering(raw, phase_codes_filename='/Users/abroome/Documents/SDR/uhd_radar/phases_1M.bin')


In [ ]:
raw.config['GENERATE']['sample_rate']

### Enter processing parameters

In [ ]:
#zero_sample_idx = 36 # X310, fs = 20 MHz
zero_sample_idx = 63 # X310, fs = 50 MHz
#zero_sample_idx = 81 # X310, fs = 100 MHz ????????
#zero_sample_idx = 159 # B205mini, fs = 56 MHz

nstack = 1 #len(raw.pulse_idx) # number of pulses to stack

modify_rx_window = False # set to true if you want to window the reference chirp only on receive, false uses ref chirp as transmitted in config file
rx_window = "kaiser14" # what you want to change the rx window to if modify_rx_window is true

dielectric_constant = 3.17 # ice (air = 1, 66% velocity coax = 2.2957)
sig_speed = scipy.constants.c / np.sqrt(dielectric_constant)

### Generate reference chirp

In [ ]:
if modify_rx_window:
    config = copy.deepcopy(raw.config)
    config['GENERATE']['window'] = rx_window
else:
    config = raw.config
    
# config = copy.deepcopy(raw.config)
# config['GENERATE']['lo_offset_sw'] = 12.5e6

chirp_ts, ref_chirp = generate_chirp(config)

In [ ]:
fig, axs = plt.subplots(2,1)

# Time domain plot
axs[0].plot(chirp_ts*1e6, np.real(ref_chirp), label='I')
axs[0].plot(chirp_ts*1e6, np.imag(ref_chirp), label='Q')
axs[0].set_xlabel('Time [us]')
axs[0].set_ylabel('Samples')
axs[0].set_title('TX Chirp -- Time Domain')
axs[0].legend()

# Frequency domain plot
freqs = scipy.fft.fftshift(scipy.fft.fftfreq(ref_chirp.size, d=1/raw.config['GENERATE']['sample_rate']))
#freqs += 330e6
ms = 20*np.log10(scipy.fft.fftshift(np.abs(scipy.fft.fft(ref_chirp))))
axs[1].plot(freqs/1e6, ms)
axs[1].set_xlabel('Frequency [MHz]')
axs[1].set_ylabel('Power [dB]')
axs[1].set_title('TX Chirp -- Frequency Domain')
axs[1].grid()

fig.tight_layout()

plt.show()
#fig.savefig(f"/Users/abroome/Desktop/Summit2023/figs/{raw.basename}_tx_timedomain.png")

### Filter reference chirp

In [ ]:
filter = signal.butter(1, 0.5e6, btype='highpass', output='sos', fs=config['GENERATE']['sample_rate'])

ref_chirp = signal.sosfilt(filter, ref_chirp)

### View raw pulse in time domain to check for clipping

In [ ]:
#plot1 = np.real(raw.radar_data[:,0]).hvplot.line(x='fast_time', label='Real') * np.imag(raw.radar_data[:,0]).hvplot.line(x='fast_time', label='Imaginary')
plot1 = np.real(raw.radar_data[:,482500]).hvplot.line(x='fast_time', label='Real') * np.imag(raw.radar_data[:,482500]).hvplot.line(x='fast_time', label='Imaginary')

plot1 = plot1.opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude')
plot1

In [ ]:
plot1 = np.real(raw.radar_data[:,490000]).hvplot.line(x='fast_time', label='Real') * np.imag(raw.radar_data[:,490000]).hvplot.line(x='fast_time', label='Imaginary')

plot1 = plot1.opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude')
plot1

In [ ]:
idx1 = 482500
idx2 = 490000

plot1 = np.real(raw.radar_data[:,idx1] - raw.radar_data[:,idx2]).hvplot.line(x='fast_time', label='Real Diff') * np.imag(raw.radar_data[:,idx1] - raw.radar_data[:,idx2]).hvplot.line(x='fast_time', label='Imaginary Diff')

plot1 = plot1.opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude', title=f"{raw.basename} pulse_idx: {idx1} - {idx2}")
plot1

In [ ]:
hvplot.save(plot1, f'/Users/abroome/Desktop/Summit2023/figs/{raw.basename}_pulse_idxs_{idx1}_{idx2}.html')

In [ ]:
#subsampled_data = raw["radar_data"][:,::100]
real_max = xr.apply_ufunc(
                lambda x: np.abs(np.real(x)),
                raw,
                dask="parallelized"
            ).max(dim='sample_idx')
imag_max = xr.apply_ufunc(
                lambda x: np.abs(np.imag(x)),
                raw,
                dask="parallelized"
            ).max(dim='sample_idx')
gain_plot = real_max.hvplot.line(label='Abs max of real part') * imag_max.hvplot.line(label='Abs max of imag part')
gain_plot.opts(title=f'{raw.basename}', ylabel='Amplitude')
gain_plot

### Clean and stack data

In [ ]:
#stacked = pr.fill_errors(raw, error_fill_value=0.0) # fill receiver errors with 0s

stacked = pr.stack(raw, nstack) # stack 

In [ ]:
stacked

### Filter LO out of received data

In [ ]:
filt_hp = signal.butter(1, 0.5e6, btype='highpass', output='sos', fs=config['GENERATE']['sample_rate'])

filtered = xr.apply_ufunc(
    lambda x: signal.sosfilt(filt_hp, x),
    stacked,
    dask="parallelized"
)

#filtered = signal.sosfilt(filt_hp, stacked.radar_data)

# output_len = raw["radar_data"].shape[0]
# travel_time = np.linspace(0, output_len/config['GENERATE']['sample_rate'], output_len)
# travel_time = travel_time - travel_time[zero_sample_idx]

# coords = {"travel_time": travel_time}
# if sig_speed is not None:
#     coords['reflection_distance'] = ("travel_time", travel_time * (sig_speed/2))

# filtered = xr.apply_ufunc(
#     lambda x: scipy.sosfilt(filt_hp, x),
#     stacked,
#     input_core_dims=[['sample_idx']], # The dimension operated over -- aka "don't vectorize over this"
#     #output_core_dims=[["travel_time"]], # The output dimensions of the lambda function itself
#     exclude_dims=set(("sample_idx",)), # Dimensions to not vectorize over
#     vectorize=True, # Vectorize other dimensions using a call to np.vectorize
#     dask="parallelized", # Allow dask to chunk and parallelize the computation
#     output_dtypes=[raw["radar_data"].dtype], # Needed for dask: explicitly provide the output dtype
#     #dask_gufunc_kwargs={"output_sizes": {'travel_time': output_len}} # Also needed for dask:
#     # explicitly provide the output size of the lambda function. See
#     # https://docs.dask.org/en/stable/generated/dask.array.gufunc.apply_gufunc.html
# ).assign_coords(coords) # And finally add coordinate(s) corresponding to the new "travel_time" dimension

# # Save the input parameters for future reference
# filtered.attrs["filter_lo"]={
#         "fs": config['GENERATE']['sample_rate'], "chirp": ref_chirp, "zero_sample_idx": zero_sample_idx, "signal_speed": sig_speed}

In [ ]:
freqs_filt, h = signal.sosfreqz(filt_hp, fs=config['GENERATE']['sample_rate'])

# Plot magnitude response of the filter
fig = plt.figure()
plt.plot(freqs_filt/(1e6), 20 * np.log10(abs(h)),
         'r', label='Bandpass filter', linewidth='2')
plt.xlabel('Frequency [MHz]', fontsize=20)
plt.ylabel('Magnitude [dB]', fontsize=20)
plt.title('Filter', fontsize=20)
plt.grid()

### Pulse compress data

In [ ]:
compressed = pr.pulse_compress(stacked, ref_chirp,
                               fs=stacked.config['GENERATE']['sample_rate'],
                               zero_sample_idx=zero_sample_idx,
                               signal_speed=sig_speed)

compressed_power = xr.apply_ufunc(
    lambda x: 20*np.log10(np.abs(x)),
    compressed,
    dask="parallelized"
)

### View 1D pulse compressed data

In [ ]:
plot1D = compressed_power.radar_data[0,:].compute().hvplot.line()
#plot1D = plot1D * compressed_power.radar_data[0,:].hvplot.line()
# relevant options: xlim(-80,1000)

plot1D = plot1D.opts(xlabel='Reflection Distance (m)', ylabel='Return Power (dB)', title=raw.basename)
plot1D

In [ ]:
import scipy.io

data_dict = {"pc": compressed.radar_data, "dist": compressed.reflection_distance}

scipy.io.savemat(f"outputs/{raw.basename}_pc.mat", mdict=data_dict)

In [ ]:
raw.basename

In [ ]:
# hvplot.extension('bokeh')
# plot1D.fig_size=(10,8)
#hvplot.save(plot1D, raw.basename+'_pulse_compressed.svg', fmt='svg')
#export_svg(plot1D, filename='/Desktop/Summit2023/figs/'+raw.basename+'_pulse_compressed.svg') # works
hvplot.save(plot1D, f'/Users/abroome/Desktop/Summit2023/figs/{raw.basename}_pulse_compressed_n{nstack}_lo_offset.html')

fname = f'/Users/abroome/Desktop/Summit2023/figs/{raw.basename}_pulse_compressed_n{nstack}_lo_offset.pickle'

with open(fname, 'wb') as f:
    pickle.dump({'plot': plot1D, 'basename': raw.basename, 'nstack': nstack, 'pc': compressed_power.radar_data[0,:].compute()}, f)

In [ ]:
# USING MATPLOTLIB
fig, ax = plt.subplots(1,1, figsize=(10,6), facecolor='white')
ax.plot(compressed_power.reflection_distance, compressed_power.radar_data[0,:])
ax.set_xlabel('Depth (m)')
ax.set_ylabel('Return Power (dB)')
ax.set_title(raw.basename)

fig.savefig(f'/Users/abroome/Desktop/Summit2023/figs/{raw.basename}_pulse_compressed_n{nstack}.png', dpi=300)

### View 2D pulse compressed data (radargram)

In [ ]:
# USING HOLOVIEWS (sometimes breaks)
plot2D = compressed_power.swap_dims({'pulse_idx': 'slow_time', 'travel_time': 'reflection_distance'}).hvplot.quadmesh(x='slow_time', cmap='inferno', ylim=(-50,1500), flip_yaxis=True)
# relevant options: ylim=(100,-50), clim=(-90,-40)

plot2D.opts(xlabel='Slow Time (s)', ylabel='Depth (m)', clabel='Return Power (dB)')
plot2D

In [ ]:
# USING MATPLOTLIB (sometimes takes a while)
fig, ax = plt.subplots(1,1, figsize=(10,6), facecolor='white')

p = ax.pcolormesh(compressed_power.slow_time, compressed_power.reflection_distance, compressed_power.radar_data.transpose(), shading='auto', cmap='inferno')
ax.invert_yaxis()
clb = fig.colorbar(p, ax=ax)
clb.set_label('Return Power (dB)')
ax.set_xlabel('Slow Time (s)')
ax.set_ylabel('Distance to Reflector (m)')
ax.set_title(f'{raw.basename}: nstack={nstack}')
# relevant options: ax.set_ylim(100,-50), ax.set_xlim(0, 1), vmin=-90, vmax=40
ax.set_ylim(1500,-50)

### View spectrogram of stacked data

In [ ]:
inpt = raw
inpt["radar_data"].shape

In [ ]:
# data = stacked["radar_data"].to_numpy()
n = 1 #len(raw.pulse_idx)
idx = 339630
pulse = pr.stack(inpt, n)[{'pulse_idx':idx}]["radar_data"].to_numpy()

f, t, S = scipy.signal.spectrogram(
    pulse,
    fs=raw.attrs["config"]["GENERATE"]["sample_rate"],
    window='flattop',
    nperseg=20,
    scaling='density', mode='psd',
    return_onesided=False
)

In [ ]:
fig, ax = plt.subplots(facecolor='white', figsize=(10,6))
freq_mhz = (np.fft.fftshift(f) + raw.attrs['config']['RF0']['freq']) / 1e6
pcm = ax.pcolormesh(t, freq_mhz, 10*np.log10(np.abs(np.fft.fftshift(S, axes=0))), shading='nearest') #  vmin=-420, vmax=-200
clb = fig.colorbar(pcm, ax=ax)
clb.set_label('Power [dB]')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Frequency [MHz]')
#ax.set_title(f"Spectrogram of received data with n_stack={n}")
ax.set_title(f"Spectrogram of received data at pulse_idx={idx}")

In [ ]:
fig.savefig(f"/Users/abroome/Desktop/Summit2023/figs/{raw.basename}-spectrogram-ft-pulse_idx{idx}_n{n}.png")

In [ ]:
import pickle
with open(f"outputs/{raw.basename}_spectrogram_ft.pickle", "wb") as f:
    pickle.dump({'f': freq_mhz, 't': t, 'spectrogram': S, 'prefix': raw.basename, 'n_stack': n, 'pulse_idx': idx}, f)

### View Power Spectrum of All Received Data

In [ ]:
single_stack = pr.stack(raw, raw.radar_data.shape[1])

data_tx_fft = da.fft.fft(ref_chirp, n=raw.radar_data.shape[0]) / raw.radar_data.shape[0]
data_rx_fft = da.fft.fft(raw.radar_data, axis=0) / raw.radar_data.shape[0]
stacked_fft = da.fft.fft(stacked.radar_data, axis=0) / stacked.radar_data.shape[0]
full_fft = da.fft.fft(single_stack.radar_data, axis=0) / single_stack.radar_data.shape[0]

data_tx_fft_pwr = 20*da.log10(da.abs(data_tx_fft))
data_rx_fft_pwr = 20*da.log10(da.abs(data_rx_fft))
stacked_fft_pwr = 20*da.log10(da.abs(stacked_fft))
full_fft_pwr = 20*da.log10(da.abs(full_fft))

#data_rx_fft_pwr.shape

In [ ]:
raw.radar_data.shape

In [ ]:
data_tx_fft = da.fft.fft(ref_chirp, n=raw.radar_data.shape[0]) / raw.radar_data.shape[0]
#data_rx_fft = da.fft.fft(filtered.radar_data, axis=0) / filtered.radar_data.shape[0]
#filtered_fft = np.fft.fft(filtered) / filtered.shape[0]
stacked_fft = da.fft.fft(stacked.radar_data, axis=0) / stacked.radar_data.shape[0]

data_tx_fft_pwr = 20*da.log10(da.abs(data_tx_fft))
#data_rx_fft_pwr = 20*da.log10(da.abs(data_rx_fft))
#filtered_fft_pwr = 20*np.log10(np.abs(filtered_fft))
stacked_fft_pwr = 20*da.log10(da.abs(stacked_fft))


In [ ]:
normal_pulse_fft = np.fft.fftshift(20*np.log10(np.abs(np.fft.fft(raw.radar_data[:,339630]))))
weird_pulse_fft = np.fft.fftshift(20*np.log10(np.abs(np.fft.fft(raw.radar_data[:,339631]))))

fig, axs = plt.subplots(facecolor='white', figsize=(10,6))
freqs = np.fft.fftshift(np.fft.fftfreq(normal_pulse_fft.shape[0], d=1/raw.config['GENERATE']['sample_rate']))
axs.plot(freqs/1e6, normal_pulse_fft, label='Normal Pulse')
axs.plot(freqs/1e6, weird_pulse_fft, label='Weird Pulse')
axs.set_xlabel('Frequency [MHz]')
axs.set_ylabel('Power [dB]')
axs.set_title('Spectrum -- Power')
axs.grid()
axs.legend()

In [ ]:
# fig, axs = plt.subplots(2,1)
fig, axs = plt.subplots(facecolor='white', figsize=(10,6))
freqs = np.fft.fftshift(np.fft.fftfreq(data_rx_fft_pwr.shape[0], d=1/raw.config['GENERATE']['sample_rate']))
axs.plot(freqs/1e6, np.fft.fftshift(data_tx_fft_pwr), label='Transmitted Pulse')
axs.plot(freqs/1e6, np.fft.fftshift(data_rx_fft_pwr[:,0]), label='Single Pulse')
axs.plot(freqs/1e6, np.fft.fftshift(stacked_fft_pwr[:,0]), label=f'Single Stack: n = {nstack}')
#axs.plot(freqs/1e6, np.fft.fftshift(filtered_fft_pwr), label='filtered')
axs.plot(freqs/1e6, np.fft.fftshift(full_fft_pwr[:,0]), label=f'Full File: n = {raw.radar_data.shape[1]}')
axs.set_xlabel('Frequency [MHz]')
axs.set_ylabel('Power [dB]')
axs.set_title(f'{raw.basename} Spectrum -- Power')
axs.grid()
axs.legend()

# axs[1].plot(freqs/1e6, np.fft.fftshift(np.angle(data_rx_fft[:,0])))
# axs[1].plot(freqs/1e6, np.fft.fftshift(np.angle(stacked_fft[:,0])))
# axs[1].plot(freqs/1e6, np.fft.fftshift(np.angle(full_fft[:,0])))
# axs[1].set_xlabel('Frequency [MHz]')
# axs[1].set_ylabel('Phase [rad]')
# axs[1].set_title('Spectrum -- Phase')
# axs[1].grid()
# fig.tight_layout()

In [ ]:
fig.savefig(f"/Users/abroome/Desktop/Summit2023/figs/{raw.basename}_rx_spectrum.png")

In [ ]:
files = []
files.append("/Users/abroome/Desktop/Summit2023/figs/20230725_161051_pulse_compressed_n987265_rectrx.pickle")
files.append("/Users/abroome/Desktop/Summit2023/figs/20230725_161051_pulse_compressed_n987265_hammingrx.pickle")
files.append("/Users/abroome/Desktop/Summit2023/figs/20230725_161051_pulse_compressed_n987265_kaiser14rx.pickle")

plots = []

for file in files:
    with open(file, "rb") as f:
        data = pickle.load(f)
        plots.append(data['plot'])

In [ ]:
plot1 = hv.Overlay(plots)
plot1.opts(xlabel='Reflection Distance (m)', ylabel='Return Power (dB)', width=500, height=400)

In [ ]:
delta_dsp = (200e6/(2**32))
delta_rf = (20e6/200)

In [ ]:
12.5e6 / delta_dsp

In [ ]:
330e6 + 12.5e6